<a href="https://colab.research.google.com/github/GOTWIC/AI-Chess-Engine/blob/main/Chess_Evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [30]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

## Import Dataset 

1.   Download Kaggle API .json file
2.   Upload to Google Drive (root folder)
3.   Mount Google Drive



In [ ]:
#Run the following commented lines if you haven't ever connected Kaggle and Google Drive before

#! pip install kaggle
#! mkdir ~/.kaggle
#! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
#! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download ronakbadhe/chess-evaluations
! unzip chess-evaluations.zip

## Load and Read Dataset

In [59]:
path = 'chessData.csv' 
rawData = pd.read_csv(path)

## Parse FEN + Evaluation into Custom Data Frame

The original data frame includes only two columns: the FEN and the Evaluation. To prepare the dataset for inputting, we need to reformat the input. The reformatting parses the FEN string and allocates a column for every square on the board (64 squares), and an additional column for the evaluation (65 columns in total).

In [105]:
def parsePiece(piece):
  return {
        'P': 1,
        'N': 2,
        'B': 3,
        'R': 4,
        'Q': 5,
        'K': 6,
        'p': -1,
        'n': -2,
        'b': -3,
        'r': -4,
        'q': -5,
        'k': -6,
    }[piece]

print(rawData.FEN[0])
print(rawData.FEN[1])

rows = 2 #len(rawData)
rawParse = np.empty([rows, 65])

for row in range(rows):
  FENIndex = 0
  for char in rawData.FEN[row]:
    if char.isalpha():
      rawParse[row][FENIndex] = parsePiece(char)
      FENIndex += 1
    elif char.isdigit():
      for emptySpace in range(int(char)):
        rawParse[row][FENIndex] = 0
        FENIndex += 1
    elif char == ' ':
      break

  rawParse[row][64] = int(rawData.Evaluation[row])

squareLabels = []
for i in range(1, 9):
  for j in range(1, 9):
    squareLabels.append(chr(j + 96) + chr((8-i) + 49))
squareLabels.append('Evaluation')

data = pd.DataFrame(rawParse, columns = squareLabels)

data.head()



rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1
rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2


,a8,b8,c8,d8,e8,f8,g8,h8,a7,b7,...,h2,a1,b1,c1,d1,e1,f1,g1,h1,Evaluation
0,-4.0,-2.0,-3.0,-5.0,-6.0,-3.0,-2.0,-4.0,-1.0,-1.0,...,1.0,4.0,2.0,3.0,5.0,6.0,3.0,2.0,4.0,-10.0
1,-4.0,-2.0,-3.0,-5.0,-6.0,-3.0,-2.0,-4.0,-1.0,-1.0,...,1.0,4.0,2.0,3.0,5.0,6.0,3.0,2.0,4.0,56.0
